## Задание

- Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
- Cохранить результаты в pandas dataframe

---

Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных: 
 * Привести колонки датафрейма к одному типу данных 
 * Проверить индекс, чтобы он соответствовал порядку строк 
 * Проверить наличие пропусков и в случае обнаружения, определить стратегию их обработки

---

Провести мини-исследование рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов. 
+ Рассчитать средние цены за квадратный метр по регионам. Важно чтобы пропуски не учитывались при расчете средней цены. 
+ Рассчитать объемы вводимого жилья по регионам и по годам. 
+ Сравнить этажность домов, количество квартир, жилую площадь, количество парковочных мест и цену за квадратный метр в разных регионах 
+ Визуализировать разброс, связь и распределение этих показателей по регионам 
+ Сделать выводы по каждому из расчетов и графиков

### Загрузка библиотек

In [1]:
import requests
import pandas as pd

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 30
pd.options.display.precision = 2
pd.options.display.show_dimensions = True

### Сбор информации

In [3]:
# Получение списка объектов

limit_ = 1000
all_obj_ids = []
for offset_ in range(0, 12000, 1000):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    response = requests.get(url)
    objects_data = response.json()
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    all_obj_ids.extend(objids)

In [15]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/36774'
requests.get(url).json()

{'data': {'id': 36774,
  'pdId': 351714,
  'developer': {'devId': 6926,
   'devShortCleanNm': 'ОКЕАН',
   'devShortNm': 'ОКЕАН',
   'devFullCleanNm': 'ОКЕАН',
   'problObjCnt': 0,
   'buildObjCnt': 1,
   'comissObjCnt': 0,
   'regRegionDesc': 'Башкортостан Республика',
   'devPhoneNum': '+7(927)300-31-01',
   'devSite': 'altun-yar.ru',
   'devEmail': 'khabibovr@mail.ru',
   'devInn': '0277093608',
   'devOgrn': '1080277004521',
   'devKpp': '027701001',
   'devLegalAddr': 'Башкортостан Республика, город Уфа, проспект Проспект Октября дом 85',
   'devFactAddr': 'Респ Башкортостан Город Уфа Проспект Октября д. 85',
   'lastRpdId': 511041,
   'fundGuarantyFlg': 0,
   'devOrgRegRegionCd': 2,
   'devEmplMainFullNm': 'Хабибов Рамиль Хамидуллович',
   'developerGroupName': 'Океан',
   'orgForm': {'id': 4,
    'fullForm': 'Общество с ограниченной ответственностью',
    'shortForm': 'ООО'},
   'companyGroupId': 7250,
   'objGuarantyEscrowFlg': 1,
   'govFundFlg': 0},
  'region': 2,
  'address':

In [4]:
# Сбор информации по объектам

all_objects_description = []

for id in all_obj_ids:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
    response_object = requests.get(url)
    obj = response_object.json()
    all_objects_description.append(obj)

In [5]:
# Сохранение информации в Pandas DataFrame

data = pd.DataFrame.from_dict(pd.json_normalize(all_objects_description), orient='columns')


In [6]:
data.shape

(10527, 109)

In [8]:
data.head()

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,data.developer.regRegionDesc,data.developer.devPhoneNum,data.developer.devSite,data.developer.devEmail,data.developer.devInn,data.developer.devOgrn,data.developer.devKpp,data.developer.devLegalAddr,data.developer.devFactAddr,data.developer.lastRpdId,data.developer.fundGuarantyFlg,data.developer.devOrgRegRegionCd,data.developer.devEmplMainFullNm,data.developer.orgForm.id,data.developer.orgForm.fullForm,data.developer.orgForm.shortForm,data.developer.objGuarantyEscrowFlg,data.developer.govFundFlg,data.region,data.address,data.objElemLivingCnt,data.objReady100PercDt,data.wallMaterialShortDesc,data.objLkFinishTypeDesc,data.objLkFreePlanDesc,data.objElemParkingCnt,data.objSquareLiving,data.rpdNum,data.rpdPdfLink,data.rpdIssueDttm,data.objLkLatitude,data.objLkLongitude,data.objCompensFundFlg,data.objProblemFlg,data.objLkClassDesc,data.loadDttm,data.photoRenderDTO,data.objGuarantyEscrowFlg,data.objectType,data.residentialBuildings,data.newBuildingId,data.objFlatCnt,data.objFlatSq,data.objNonlivElemCnt,data.objStatus,data.isAvailableWantLiveHere,data.objInfrstrBicycleLaneFlg,data.objInfrstrRampFlg,data.objInfrstrCurbLoweringFlg,data.objectTransportInfo,data.conclusion,data.objGreenHouseFlg,data.objEnergyEffShortDesc,data.infrastructureIndexValue,data.objLkSaleDepEmail,data.developer.developerGroupName,data.developer.companyGroupId,data.floorMin,data.floorMax,data.miniUrl,data.objFloorCnt,data.quartography.objLivElem1KCnt,data.quartography.objLivElem2KCnt,data.quartography.objLivElem3KCnt,data.quartography.objLivElem4KCnt,data.quartography.objLivElemCnt,data.quartography.objLivElemEntrCnt,data.quartography.objLivElemEntrFloorAvg,data.objTransferPlanDt,data.objLivCeilingHeight,data.objInfrstrPlaygrndCnt,data.objInfrstrSportGroundCnt,data.objInfrstrTrashAreaCnt,data.objInfrstrObjPrkngCnt,data.objInfrstrNotObjPrkngCnt,data.objElevatorWheelchairCnt,data.objElevatorPassengerCnt,data.objElevatorCargoCnt,data.soldOutPerc,data.objPriceAvg,data.generalContractorNm,data.nonlivFirstFloor,data.transportDistIndex,data.transportDistIndexValue,data.nameObj,data.greenAreaIndexValue,data.airQualityIndexValue,data.complexShortNm,data.developer.bankruptStage.bankruptStageCd,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm,data.metro.id,data.metro.name,data.metro.line,data.metro.color,data.metro.time,data.metro.isWalk,data.metro.colors
0,0,29679,124647,7424,212-2,212-2,212-2,0,2,0,Ростовская область,+7(961)285-90-10,taganrogjsk.wixsite.com,kirpol76@gmail.com,6154572922,1116154005923,615401001,"Ростовская область, город Таганрог, улица Своб...",обл Ростовская город Таганрог улица Сергея Шил...,126093.0,0,61.0,Снимщикова Ольга Викторовна,20,Жилищный или жилищно-строительный кооператив,ЖСК,0,0,61,г Таганрог,18,2020-03-31,Другое,Под ключ,Есть,0,1519.00,61-000818,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,20-03-2019 18:41,47.2238,38.8513,0,0,Комфорт,16-08-2022 01:17,[],0,normal,1,40800,18,1519.0,0,0,True,0,0,0,[],False,0,Не нормируется,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,29678,124648,7424,212-2,212-2,212-2,0,2,0,Ростовская область,+7(961)285-90-10,taganrogjsk.wixsite.com,kirpol76@gmail.com,6154572922,1116154005923,615401001,"Ростовская область, город Таганрог, улица Своб...",обл Ростовская город Таганрог улица Сергея Шил...,126093.0,0,61.0,Снимщикова Ольга Викторовна,20,Жилищный или жилищно-строительный кооператив,ЖСК,0,0,61,"город Таганрог, улица Сергея Шило, д. 212-2",27,2020-03-31,Другое,Под чистовую,Нет,0,1519.00,61-000819,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,20-03-2019 18:18,47.2197,38.9134,0,0,Комфор

In [13]:
data.iloc(axis=1)[0:40].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10527 entries, 0 to 10526
Data columns (total 40 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   errcode                              10527 non-null  object 
 1   data.id                              10527 non-null  int64  
 2   data.pdId                            10527 non-null  int64  
 3   data.developer.devId                 10527 non-null  int64  
 4   data.developer.devShortCleanNm       10527 non-null  object 
 5   data.developer.devShortNm            10527 non-null  object 
 6   data.developer.devFullCleanNm        10527 non-null  object 
 7   data.developer.problObjCnt           10527 non-null  int64  
 8   data.developer.buildObjCnt           10527 non-null  int64  
 9   data.developer.comissObjCnt          10527 non-null  int64  
 10  data.developer.regRegionDesc         10407 non-null  object 
 11  data.developer.devPhoneNum  